# this will setup a directory of files that can be rcp'd to raijin. On raijin then run RsyncGitHub_toStage_raijin.ipynb to transfer to snappper then github.

#try to think of a system where if a new file appears that isnt in cloned github, there will be a message.

In [2]:
print('BEGIN')

import os
import shutil
import glob
import sys
import inspect
import filecmp
from paramiko import SSHClient
from scp import SCPClient

CRED = '\033[91m'
CEND = '\033[0m'

__file__='jupyter_notebook' #this can be deleted when written to a python script and loaded as module.

files_to_scp=[]

workdir='/OSM/CBR/OA_DCFP/work/col414/cafepp' #location of working code to compare with...
topdir='/OSM/CBR/OA_DCFP/apps/col414'         #location of clean code to compare with

os.chdir(topdir)

print('Current Working Directory=',os.getcwd())

if(not os.path.exists(topdir+'/'+'rsync_stage')):
  os.mkdir(topdir+'/'+'rsync_stage')

clean_cafepp_directory=topdir+'/'+'rsync_stage'+'/'+'cafepp_clean'

if(os.path.exists(clean_cafepp_directory)):
  print(clean_cafepp_directory,' file exists, need to move it to a unique directory...')
  
  number_searches=100 #choose big number later e.g. 1000

  for n in range(number_searches):

    backup_clean_cafepp_directory=topdir+'/'+'rsync_stage'+'/'+'cafepp_clean.old'+str(n)
    print('n,backup_clean_cafepp_directory=',n,backup_clean_cafepp_directory)
    if(not os.path.exists(backup_clean_cafepp_directory)):
      print('Found unique backup_clean_cafepp_directory=',backup_clean_cafepp_directory)
      break
  
  print('Move '+clean_cafepp_directory+' to '+backup_clean_cafepp_directory)

  shutil.move(clean_cafepp_directory,backup_clean_cafepp_directory)
  
os.chdir(topdir+'/'+'rsync_stage')

print('Current Working Directory=',os.getcwd())

print('Running git...')

os.system('git clone https://github.com/markcollier/cafepp.git')

shutil.move(topdir+'/'+'rsync_stage'+'/'+'cafepp',clean_cafepp_directory)

suffixes=['py','ipynb','json','bash','csh']

#top directory:
for suffix_cnt,suffix in enumerate(suffixes):
  files=glob.glob(clean_cafepp_directory+'/'+'*.'+suffix)
  print('files=',files)
  for file in files:
    print('file=',file)
    head,tail=os.path.split(file)
    #print(head,tail)
    
    if(not os.path.exists(workdir+'/'+tail)):
      print(CRED+workdir+'/'+tail+' missing from source directory...'+CEND)
      
    if(filecmp.cmp(file,workdir+'/'+tail)):
      print(file +' and '+workdir+'/'+tail+' same.')
    else:
      print(CRED+file +' and '+workdir+'/'+tail+' different.'+CEND)
      files_to_scp.append(tail)

#raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))

#sub-directory
sub_directory='JsonTemplates'
for suffix_cnt,suffix in enumerate(suffixes):
  files=glob.glob(clean_cafepp_directory+'/'+sub_directory+'/'+'*.'+suffix)
  for file in files:
    head,tail=os.path.split(file)
    if(not os.path.exists(workdir+'/'+sub_directory+'/'+tail)):
      print(CRED+workdir+'/'+sub_directory+'/'+tail+' missing from source directory...'+CEND)
    if(filecmp.cmp(file,workdir+'/'+sub_directory+'/'+tail)):
      print(file +' and '+workdir+'/'+tail+' same.')
    else:
      print(CRED+file +' and '+workdir+'/'+sub_directory+'/'+tail+' different.'+CEND)
      files_to_scp.append(sub_directory+'/'+tail)

#sub-directory
sub_directory='TablesTemplates'
for suffix_cnt,suffix in enumerate(suffixes):
  files=glob.glob(clean_cafepp_directory+'/'+sub_directory+'/'+'*.'+suffix)
  for file in files:
    head,tail=os.path.split(file)
    if(not os.path.exists(workdir+'/'+sub_directory+'/'+tail)):
      print(CRED+workdir+'/'+sub_directory+'/'+tail+' missing from source directory...'+CEND)
    if(filecmp.cmp(file,workdir+'/'+sub_directory+'/'+tail)):
      print(file +' and '+workdir+'/'+tail+' same.')
    else:
      print(CRED+file +' and '+workdir+'/'+sub_directory+'/'+tail+' different.'+CEND)
      files_to_scp.append(sub_directory+'/'+tail)
      
#raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))

os.chdir(topdir)

print('Current Working Directory=',os.getcwd())
      
print('Files to scp from vm32 to raijin are:',' '.join(files_to_scp))

#raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))

# #os.system('scp '+' '.join(files_to_scp)+' mac599@raijin.nci.org.au:/short/v14/mac599/rsync_stage/to_merge')

# print('cd to '+workdir+' and run this command from there: scp '+' '.join(files_to_scp)+' mac599@raijin.nci.org.au:/short/v14/mac599/rsync_stage/to_merge')

ssh = SSHClient()
ssh.load_system_host_keys()
ssh.connect(hostname='raijin.nci.org.au',username='mac599',password='Kluger_09h') #,password='Kluger_09h' comment out password on github or use a key somehow.

# SCPCLient takes a paramiko transport as an argument
scp = SCPClient(ssh.get_transport())

for infile in files_to_scp:
  print(CRED+'Copying '+workdir+'/'+infile+' to '+'/short/v14/mac599/rsync_stage/to_merge'+'/'+infile+CEND)
  scp.put(workdir+'/'+infile, '/short/v14/mac599/rsync_stage/to_merge'+'/'+infile)
  #scp.get('README.txt')

# Uploading the 'test' directory with its content in the
# '/home/user/dump' remote directory
#scp.put('test', recursive=True, remote_path='/home/user/dump')

scp.close()

print('Got to raijin and merge this files into source area...')
      
print('END')

BEGIN
Current Working Directory= /OSM/CBR/OA_DCFP/apps/col414
/OSM/CBR/OA_DCFP/apps/col414/rsync_stage/cafepp_clean  file exists, need to move it to a unique directory...
n,backup_clean_cafepp_directory= 0 /OSM/CBR/OA_DCFP/apps/col414/rsync_stage/cafepp_clean.old0
n,backup_clean_cafepp_directory= 1 /OSM/CBR/OA_DCFP/apps/col414/rsync_stage/cafepp_clean.old1
n,backup_clean_cafepp_directory= 2 /OSM/CBR/OA_DCFP/apps/col414/rsync_stage/cafepp_clean.old2
n,backup_clean_cafepp_directory= 3 /OSM/CBR/OA_DCFP/apps/col414/rsync_stage/cafepp_clean.old3
n,backup_clean_cafepp_directory= 4 /OSM/CBR/OA_DCFP/apps/col414/rsync_stage/cafepp_clean.old4
n,backup_clean_cafepp_directory= 5 /OSM/CBR/OA_DCFP/apps/col414/rsync_stage/cafepp_clean.old5
n,backup_clean_cafepp_directory= 6 /OSM/CBR/OA_DCFP/apps/col414/rsync_stage/cafepp_clean.old6
n,backup_clean_cafepp_directory= 7 /OSM/CBR/OA_DCFP/apps/col414/rsync_stage/cafepp_clean.old7
n,backup_clean_cafepp_directory= 8 /OSM/CBR/OA_DCFP/apps/col414/rsync_stage/c

Copying /OSM/CBR/OA_DCFP/work/col414/cafepp/cafepp_daily.py to /short/v14/mac599/rsync_stage/to_merge/cafepp_daily.py
Copying /OSM/CBR/OA_DCFP/work/col414/cafepp/cafepp_daily_forecast.py to /short/v14/mac599/rsync_stage/to_merge/cafepp_daily_forecast.py
Copying /OSM/CBR/OA_DCFP/work/col414/cafepp/n_data_funcs.py to /short/v14/mac599/rsync_stage/to_merge/n_data_funcs.py
Copying /OSM/CBR/OA_DCFP/work/col414/cafepp/decadal_diag.py to /short/v14/mac599/rsync_stage/to_merge/decadal_diag.py
Copying /OSM/CBR/OA_DCFP/work/col414/cafepp/cafepp.py to /short/v14/mac599/rsync_stage/to_merge/cafepp.py
Copying /OSM/CBR/OA_DCFP/work/col414/cafepp/cafepp_monthly_forecast.py to /short/v14/mac599/rsync_stage/to_merge/cafepp_monthly_forecast.py
Copying /OSM/CBR/OA_DCFP/work/col414/cafepp/cafepp_daily_nino34_from_sst.ipynb to /short/v14/mac599/rsync_stage/to_merge/cafepp_daily_nino34_from_sst.ipynb
Copying /OSM/CBR/OA_DCFP/work/col414/cafepp/cafepp_monthly_nino34_from_sst.ipynb to /short/v14/mac599/rsync_

SCPException: scp: /short/v14/mac599/rsync_stage/to_merge/JsonTemplates/cafepp_vars.json: No such file or directory


In [20]:
print('BEGIN')

from paramiko import SSHClient
from scp import SCPClient

import os

os.chdir(topdir)


ssh = SSHClient()
ssh.load_host_keys(os.path.join(os.path.dirname(__file__), 'known_hosts'))

ssh.connect(hostname='raijin.nci.org.au') #,password='Kluger_09h')


#ssh.load_system_host_keys()
#ssh.connect(hostname='raijin.nci.org.au',username='mac599') #,password='Kluger_09h')

# SCPCLient takes a paramiko transport as an argument
scp = SCPClient(ssh.get_transport())

scp.put('README', 'README.txt')
#scp.get('README.txt')

# Uploading the 'test' directory with its content in the
# '/home/user/dump' remote directory
#scp.put('test', recursive=True, remote_path='/home/user/dump')

#scp.close()

print('END')

BEGIN


SSHException: No authentication methods available

In [13]:
help(SSHClient
   )

Help on class SSHClient in module paramiko.client:

class SSHClient(paramiko.util.ClosingContextManager)
 |  A high-level representation of a session with an SSH server.  This class
 |  wraps `.Transport`, `.Channel`, and `.SFTPClient` to take care of most
 |  aspects of authenticating and opening channels.  A typical use case is::
 |  
 |      client = SSHClient()
 |      client.load_system_host_keys()
 |      client.connect('ssh.example.com')
 |      stdin, stdout, stderr = client.exec_command('ls -l')
 |  
 |  You may pass in explicit overrides for authentication and server host key
 |  checking.  The default mechanism is to try to use local key files or an
 |  SSH agent (if one is running).
 |  
 |  Instances of this class may be used as context managers.
 |  
 |  .. versionadded:: 1.6
 |  
 |  Method resolution order:
 |      SSHClient
 |      paramiko.util.ClosingContextManager
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Create a new SSH